In [1]:
import json
import requests
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib             
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from datetime import datetime
from dingtalkchatbot.chatbot import DingtalkChatbot    #钉钉机器人
import smtplib
from email.header import Header
from email.mime.text import MIMEText
#设置服务器所需信息
#163邮箱服务器地址
print(matplotlib.matplotlib_fname())              #解决负号无法显示问题
# from matplotlib import font_manager 
requests.packages.urllib3.disable_warnings()
matplotlib.rc("font",family='Adobe Song Std')     #设置中文字体

C:\SoftwareSetup\Anaconda\lib\site-packages\matplotlib\mpl-data\matplotlibrc


In [4]:
def get_ex_jc(coinID,type_,jc):                       #size为K线个数，jc为设定提醒的最低门限
    url_temp_depth_qh = 'https://www.okex.com/api/swap/v3/instruments/'+str.upper(coinID)+'-'+str.upper(type_)+'-SWAP/depth?size=50'    #获取合约ticker信息
    resp_temp_depth_qh = requests.get(url_temp_depth_qh, verify = False)
    df_temp_depth_qh = resp_temp_depth_qh.json()         #获取期货数据
#     print(df_temp_depth_qh['bids'][0][0])   #合约买1价
#     print(df_temp_depth_qh['asks'][0][0])   #合约卖1价
#     print(df_temp_depth_qh)
#     print(df_temp_depth_qh['asks'])
    url_temp_tick_xh = 'https://www.okex.com/api/spot/v3/instruments/'+coinID+'-USDT/book?size=5&depth=0.001'  #现货价
    resp_temp_tick_xh = requests.get(url_temp_tick_xh, verify = False)
    if resp_temp_tick_xh.status_code == 400:
        pass;
    else:
        df_temp_tick_xh  = resp_temp_tick_xh.json()
#         print(df_temp_tick_xh['bids'][0][0])      #现货买1价
#         print(df_temp_tick_xh['asks'][0][0])      #现货卖1价
#         print(df_temp_tick_xh)
        qh_high_jc = (float(df_temp_depth_qh['bids'][0][0])-float(df_temp_tick_xh['asks'][0][0]))/float(df_temp_tick_xh['asks'][0][0])
# #         print(qh_high_jc)
        qh_low_jc = (float(df_temp_depth_qh['asks'][0][0])-float(df_temp_tick_xh['bids'][0][0]))/float(df_temp_tick_xh['bids'][0][0])
#         print(type(qh_high_jc))
#         print(type(qh_low_jc))
#         print(type(max(qh_high_jc,qh_low_jc)))
#         print(type(-float(jc)))
# #         print(qh_low_jc)
        warn =  pd.DataFrame()
        if min(qh_high_jc,qh_low_jc) > float(jc):          #价差绝对值大于一定程度触发告警
            warn = pd.DataFrame([[coinID,min(qh_high_jc,qh_low_jc),type_]],columns=['币种','价差','类型'])
            return warn
        elif max(qh_high_jc,qh_low_jc) < -float(jc):
            warn = pd.DataFrame([[coinID,max(qh_high_jc,qh_low_jc),type_]],columns=['币种','价差','类型'])
            return warn    

get_ex_jc('mask','usdt','0.0001')

,币种,价差,类型
0,mask,0.000775,usdt


In [5]:
def get_all_coin():  #获取所有币种函数
    all_coin_url = "https://www.okex.com/api/swap/v3/instruments"
    req_all_temp_coin = requests.get(all_coin_url, verify = False)
    df_all_temp_coin = pd.DataFrame(req_all_temp_coin.json())
    return df_all_temp_coin[['underlying_index','quote_currency']]
get_all_coin()

,underlying_index,quote_currency
0,OMG,USDT
1,WAVES,USDT
2,BAT,USDT
3,NEO,USDT
4,LINK,USDT
...,...,...
125,SUN,USD
126,UNI,USD
127,FIL,USD
128,GRT,USD


In [6]:
def sendemail():
    mail_host = 'smtp.qq.com'  
    #163用户名
    mail_user = '909962673@qq.com'  
    #密码(部分邮箱为授权码) 
    mail_pass = 'njomqqrpxrkdbdda'   
    #邮件发送方邮箱地址
    sender = '909962673@qq.com'  
    #邮件接受方邮箱地址，注意需要[]包裹，这意味着你可以写多个邮件地址群发
    receivers = ['909962673@qq.com'] #['361694142@qq.com','7940175@qq.com', '476125228@qq.com']#,'909962673@qq.com'],#'476125228@qq.com']  
#     content = "卖一*和真实*差值为" +"\n" \

    message = MIMEText(content,'plain','utf-8')
    if abs(warnlist['价差'].max()) >= 0.1 or abs(warnlist['价差'].min()) >= 0.1:
        message['Subject'] = Header("【OKEX】会议跟进"+str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())), 'utf-8')
    elif abs(warnlist['价差'].max()) >= 0.075 or abs(warnlist['价差'].min()) >= 0.075:
        message['Subject'] = Header("【OKEX】会议纪要"+str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())), 'utf-8')      
    elif abs(warnlist['价差'].max()) >= 0.05 or abs(warnlist['价差'].min()) >= 0.05:
        message['Subject'] = Header("【OKEX】会议准备"+str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())), 'utf-8')   
    elif abs(warnlist['价差'].max()) >= 0.025 or abs(warnlist['价差'].min()) >= 0.025:
        message['Subject'] = Header("【OKEX】会议通知"+str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())), 'utf-8')   
    else:
        message['Subject'] = Header("【OKEX】无需关注"+str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())), 'utf-8')   
    message['From'] = sender 
    #接受方信息     
    message['To'] = receivers[0]
    #登录并发送邮件
    try:
        smtpObj = smtplib.SMTP() 
        #连接到服务器
        smtpObj.connect(mail_host,25)
        #登录到服务器
        smtpObj.login(mail_user,mail_pass) 
        #发送
        smtpObj.sendmail(
        sender,receivers,message.as_string()) 
        #退出
        smtpObj.quit() 
        print('success')
    except smtplib.SMTPException as e:
        print('error',e) #打印错误

In [7]:
def dingding():          #钉钉机器人通知程序
    webhook = 'https://oapi.dingtalk.com/robot/send?access_token=f0b9ff4b13e360145cf0ae9f4ff18aa957a9e7c02973fa17e88339262c65e62b'
    qianhai = DingtalkChatbot(webhook)
    if abs(warnlist['价差'].max()) >= 0.1 or abs(warnlist['价差'].min()) >= 0.1:
        tit = '【OKEX】会议跟进'
        text_tit = '【OKEX】会议跟进:\n'+\
        '> '
    elif abs(warnlist['价差'].max()) >= 0.075 or abs(warnlist['价差'].min()) >= 0.075:
        tit = '【OKEX】会议纪要'
        text_tit = '【OKEX】会议纪要:\n'
        '> '
    elif abs(warnlist['价差'].max()) >= 0.05 or abs(warnlist['价差'].min()) >= 0.05:
        tit = '【OKEX】会议准备'
        text_tit = '【OKEX】会议准备:\n'
        '> '
    elif abs(warnlist['价差'].max()) >= 0.025 or abs(warnlist['价差'].min()) >= 0.025:
        tit = '【OKEX】会议通知'
        text_tit = '【OKEX】会议通知:\n'
        '> '
    else:
        tit = '【OKEX】暂无会议'
        text_tit = '【OKEX】暂无会议:\n'
        '> ' 
    qianhai.send_markdown(title=tit, text=text_tit+content+str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())) ,is_at_all=False)


In [8]:
def sleeptime(hour,min,sec):
    return hour*3600 + min*60 + sec;
min_ = sleeptime(0,5,0);
second = sleeptime(0,0,0.2);

In [9]:
count = 0
while 1 == 1:
    try:
        count += 1
        coin_df = pd.DataFrame()
        coin_df = get_all_coin()
        warnlist = pd.DataFrame()
        for i in range(0,len(coin_df)):
    #         if str.lower(i) == 'iota':
    #             continue;
    #         else:
            time.sleep(second)
    #         print(coin_df['underlying_index'][i])
            warn = get_ex_jc(coin_df['underlying_index'][i],coin_df['quote_currency'][i],'0.00015')  #
            warnlist = warnlist.append(warn).reset_index(drop = True)
        print(count,end='  ')
        print(str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())),end='  ')
        print('%d 个币种' %len(coin_df))
        if len(warnlist) > 0:
            warnlist = warnlist.sort_values('价差', ascending=False).reset_index(drop = True)
        else:
            time.sleep(min_);
            continue;
        #     requests.packages.urllib3.disable_warnings()
        content = ''
        for i in warnlist.index:
            if str.upper(warnlist['类型'][i]) == 'USD':
                s = warnlist['币种'][i]+'币本位永续合约价差为'+'{:.1%}'.format(warnlist['价差'][i])+'\n'
            elif str.upper(warnlist['类型'][i]) == 'USDT':     
                s = warnlist['币种'][i]+'U本位永续合约价差为'+'{:.1%}'.format(warnlist['价差'][i])+'\n'            
            else:
                s = warnlist['币种'][i]+warnlist['类型'][i]+'本位永续合约价差为'+'{:.1%}'.format(warnlist['价差'][i])+'\n'     
        #     s = print('%s 币本位永续合约价差为 %s' %(testlist['币种'][i],'{:.1%}'.format(testlist['价差'][i])))
            content += s
        if warnlist['价差'].max() >= 0.00015 or warnlist['价差'].min() <= -0.00015:
            sendemail()
            dingding()
        time.sleep(min_);
    except Exception as e:
        print(e)
        time.sleep(min_);
        continue;

1  2021-04-29 23:50:45  130 个币种
success


KeyboardInterrupt: 